In [1]:
## import csv
from datetime import datetime

## ETAPE 1 : DUPLICATION DES NOTICES ISAD INDEXEES AVEC PLUSIEURS NOMS ET SUPPRESSION DES NOTICES NON INDEXEES

In [2]:
def process_csv(input_csv_file, output_csv_file, selected_columns, column_mapping):
    new_data = []
    with open(input_csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Si la colonne 'nameAccessPoints' contient des données multiples
            if "|" in row["nameAccessPoints"]:
                # Traite les données multiples et ajoute les nouvelles lignes à la liste des nouvelles données
                new_data.extend(process_name(row, selected_columns, column_mapping))
            # Sinon, si la colonne 'nameAccessPoints' n'est pas vide
            elif row["nameAccessPoints"].strip() != "":
                # Ajoute une nouvelle ligne avec les colonnes sélectionnées à la liste des nouvelles données
                new_data.append({column_mapping.get(key, key): row.get(key, "") for key in selected_columns})
    write_csv(output_csv_file, new_data, [column_mapping.get(col, col) for col in selected_columns])

def process_name(row, selected_columns, column_mapping):
    # Divise les données multiples séparées par '|' dans la colonne 'nameAccessPoints'
    names = row["nameAccessPoints"].split("|")
    new_rows = []
    # Pour chaque nom séparé, crée une nouvelle ligne avec les colonnes sélectionnées
    for name in names:
        # Crée un nouveau dictionnaire avec les colonnes sélectionnées
        new_row = {column_mapping.get(key, key): row.get(key, "") for key in selected_columns}
        # Ajoute le nom actuel à la nouvelle ligne
        new_row[column_mapping.get("nameAccessPoints", "nameAccessPoints")] = name
        new_rows.append(new_row)
    return new_rows

def write_csv(output_csv_file, data, selected_columns):
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=selected_columns)
        writer.writeheader()
        writer.writerows(data)
    print("Fichier csv contenant uniquement les noms indexés dans", output_csv_file)
    
# LANCEMENT
# Chemins vers les fichiers csv
input_csv_file = "01_isad_data.csv"
output_csv_file = "03_isad_data_OUTPUT.csv"

# Liste des colonnes à conserver
selected_columns = ["nameAccessPoints", "title", "scopeAndContent", "eventDates", "eventStartDates", "eventEndDates", "referenceCode"]

# Dictionnaire de mappage des intitulés de colonnes
column_mapping = {
        "title": "ISAD_title",
        "scopeAndContent": "ISAD_scopeAndContent",
        "eventDates": "ISAD_eventDates",
        "eventStartDates" : "ISAD_eventStartDates",
        "eventEndDates":"ISAD_eventEndDates",
        "referenceCode":"ISAD_cote"      
}

# APPEL DE LA FONCTION
process_csv(input_csv_file, output_csv_file, selected_columns, column_mapping)


NameError: name 'csv' is not defined

## ETAPE 2 : RECUPERER LES NOUVELLES AUTORITES (false) ET EXTRAIRE CES NOMS AVEC LEUR INFORMATIONS DU FICHIER CSV 03_isad_data_OUTPUT

In [3]:
# Chemin vers le fichier CSV contenant les données initiales
initial_csv_file = "03_isad_data_OUTPUT.csv"

# Obtenez la date actuelle
current_date = datetime.now().strftime("%Y-%m-%d")

# Créer le nom du fichier CSV de sortie avec la date
output_csv_file = f"04_PHOEBUS_NEW_AUTHORITY_{current_date}.csv"

# Fonction pour extraire les lignes avec la valeur False dans une colonne donnée
def extract_false_rows(csv_file, column_name):
    false_autorites = []

    # Lecture du fichier CSV
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Parcours de chaque ligne
        for row in reader:
            # Vérification si la valeur dans la colonne spécifiée est False
            if row[column_name].strip() == "[False]":
                # Suppression littérale des séquences [' et ']
                autorite = row["Autorites"].replace("['", "").replace("']", "")
                false_autorites.append(autorite)

    return false_autorites

# Fonction pour comparer les noms extraits avec la colonne "nameAccessPoints" du fichier initial
def compare_names_with_initial(csv_file, names_to_compare):
    new_autorities = []

    # Lecture du fichier CSV initial
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Parcours de chaque ligne
        for row in reader:
            # Vérification si le nom extrait se trouve dans la colonne "nameAccessPoints"
            if row["nameAccessPoints"] in names_to_compare:
                # Ajout de la colonne supplémentaire Statut indiquant si c'est une nouvelles notices ou non
                row["Statut"] = "NEW AUTHORITY"
                new_autorities.append(row)

    return new_autorities

def write_new_authorities_to_csv(output_csv_file, new_autorities):
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
        if new_autorities:
            writer = csv.DictWriter(file, fieldnames=new_autorities[0].keys())
            writer.writeheader()
            writer.writerows(new_autorities)
            print("Les nouvelles autorités ont été écrites dans", output_csv_file)
        else:
            print("Aucun nom correspondant trouvé.")


#LANCEMENT
false_autorites = extract_false_rows("02_CLSR_Correspondance.csv", "Found")
new_autorities = compare_names_with_initial(initial_csv_file, false_autorites)
write_new_authorities_to_csv(output_csv_file, new_autorities)



NameError: name 'csv' is not defined

## ETAPE 3 : RECUPERER LES ANCIENNES AUTORITES (true) ET EXTRAIRE CES NOMS AVEC LEUR INFORMATIONS DU FICHIER CSV 03_isad_data_OUTPUT

In [11]:
# Fonction pour extraire les lignes avec la valeur True dans une colonne donnée
def extract_true_rows(csv_file, column_name):
    true_autorites = []

    # Lecture du fichier CSV
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Parcours de chaque ligne
        for row in reader:
            # Vérification si la valeur dans la colonne spécifiée est True
            if row[column_name].strip() == "[True]":
                # Suppression littérale des séquences [' et ']
                autorite = row["Autorites"].replace("['", "").replace("']", "")
                true_autorites.append(autorite)

    return true_autorites

# Fonction pour comparer les noms extraits avec la colonne "nameAccessPoints" du fichier initial
def compare_names_with_initial(csv_file, names_to_compare):
    old_autorities = []

    # Lecture du fichier CSV initial
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # Parcours de chaque ligne
        for row in reader:
            # Vérification si le nom extrait se trouve dans les noms à comparer
            if row["nameAccessPoints"] in names_to_compare:
                # Ajout de la colonne supplémentaire Statut indiquant si c'est une ancienne notice ou non
                row["Statut"] = "OLD AUTHORITY"
                old_autorities.append(row)

    return old_autorities

def write_old_authorities_to_csv(output_csv_file_old, old_autorities):
    with open(output_csv_file_old, 'w', newline='', encoding='utf-8') as file:
        if old_autorities:
            writer = csv.DictWriter(file, fieldnames=old_autorities[0].keys())
            writer.writeheader()
            writer.writerows(old_autorities)
            print("Les anciennes autorités ont été écrites dans", output_csv_file_old)
        else:
            print("Aucune ancienne autorité trouvée.")


#LANCEMENT
output_csv_file_old = f"04_PHOEBUS_OLD_AUTHORITY_{current_date}.csv"
true_autorites = extract_true_rows("02_CLSR_Correspondance.csv", "Found")
old_autorities = compare_names_with_initial(initial_csv_file, true_autorites)
write_old_authorities_to_csv(output_csv_file_old, old_autorities)


Les anciennes autorités ont été écrites dans 04_PHOEBUS_OLD_AUTHORITY_2024-05-27.csv


## APPELS DES FONCTIONS 

In [8]:
# ETAPE 1
process_csv(input_csv_file, output_csv_file, selected_columns, column_mapping)

# ETAPE 2
false_autorites = extract_false_rows("02_CLSR_Correspondance.csv", "Found")
new_autorities = compare_names_with_initial(initial_csv_file, false_autorites)
write_new_authorities_to_csv(output_csv_file, new_autorities)

# ETAPE 3
output_csv_file_old = f"04_PHOEBUS_OLD_AUTHORITY_{current_date}.csv"
true_autorites = extract_true_rows("02_CLSR_Correspondance.csv", "Found")
old_autorities = compare_names_with_initial(initial_csv_file, true_autorites)
write_old_authorities_to_csv(output_csv_file_old, old_autorities)


Les données ont été écrites dans 04_PHOEBUS_NEW_AUTHORITY_2024-05-27.csv
Aucun nom correspondant trouvé.
Les anciennes autorités ont été écrites dans 04_PHOEBUS_OLD_AUTHORITY_2024-05-27.csv
